In [2]:
from tpot import TPOTRegressor
from sklearn.linear_model import Lasso
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from matplotlib import pyplot as plt
from matplotlib import colors as mcolors
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.feature_selection import mutual_info_regression
import seaborn as sns

In [ ]:
# Preprocessing features

X_og = pd.read_pickle("cached_df.pkl")
materials = X_og["material"].tolist()
excluded_materials = ['MoS2']
X_og = X_og[[material not in excluded_materials for material in materials]]
X_og = X_og.reset_index(drop=True)
X_og.to_excel("data.xlsx")
X_og = X_og.drop(columns=["material", "formula", "structure", "composition"])
X_og = X_og.loc[:, (X_og!=0).any(axis=0)]
X_og = X_og.loc[:, X_og.nunique() > 10]
pd.set_option("display.max_columns", 400)
print(X_og.shape)

In [ ]:
# Pearson Correlation

corr = X_og.corr(method="pearson").fillna(0)
plt.figure(figsize=(30, 30))
sns.heatmap(corr, annot=False, cmap='coolwarm', fmt=".2f", linewidths=0.5, square=True)
plt.savefig("corr_plot.png", dpi=300)
plt.show()

corr = corr.stack().reset_index()
corr.columns = ['Feature1', 'Feature2', 'Correlation']
corr = corr[corr['Feature1'] < corr['Feature2']]
high_corr_pairs = corr[abs(corr['Correlation']) > 0.8]
high_corr_pairs = high_corr_pairs.sort_values(by='Correlation', ascending=False)
high_corr_pairs = high_corr_pairs.reset_index(drop=True)
feature1 = high_corr_pairs['Feature1'].values
feature2 = high_corr_pairs['Feature2'].values
from collections import Counter
counts = dict(Counter(feature1))
counts = dict(sorted(counts.items(), key=lambda item: -item[1]))
print(counts)
counts = dict(Counter(feature2))
counts = dict(sorted(counts.items(), key=lambda item: -item[1]))
print(counts)

In [ ]:
# Dropping Highly Correlated Features

X_og = X_og.drop(columns=np.unique(feature2))
print(X_og.shape)
corr = X_og.corr(method="pearson").fillna(0)
plt.figure(figsize=(20, 20))
sns.heatmap(corr, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5, square=True)
plt.savefig("corr_plot.png", dpi=300)
plt.show()

In [ ]:
# Functionalizing features

functions = {
    "squared": lambda x: x**2,
    "cubed": lambda x: x**3,
    "square_root": lambda x: abs(x)**(1/2),
    "exp": lambda x: np.exp(x),
    "log": lambda x: np.log(1+x)
}

X_func = X_og.copy()
X_copy = X_og.copy()
for key,value in functions.items():
    tmp_df = pd.DataFrame()
    if key == "exp":
        tmp_df = X_copy.apply(lambda col: np.exp(col) if col.max() <= 6 and col.min() >= -6 else col)
        for col in tmp_df.columns:
            if (tmp_df[col] == X_copy[col]).any():
                tmp_df = tmp_df.drop(columns=[col])
    elif key == "log":
        tmp_df = X_copy.apply(lambda col: np.log(1+col) if (1+col).min() > 0 else col)
        for col in tmp_df.columns:
            if (tmp_df[col] == X_copy[col]).any():
                tmp_df = tmp_df.drop(columns=[col])
    else:
        tmp_df = X_copy.map(value)
    tmp_df.columns = [f"{key}({col})" for col in tmp_df.columns]
    X_func = pd.concat([X_func, tmp_df], axis=1)
print(X_func.shape)

In [ ]:
# Scaling features

X_func = X_og
X_arr = X_func.values
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_arr)
print(X_scaled.shape)

In [ ]:
# Reading the Y values from experimental data

Y = pd.read_excel("exp_data.xlsx", header=0)
Y = Y[["System", "Rev. Cap (1C)/Th.Cap."]]
Y = Y.set_index("System")
Y = Y.loc[materials, "Rev. Cap (1C)/Th.Cap."]
Y = Y[[material not in excluded_materials for material in materials]].values
print(Y.shape)



In [ ]:
# PCA

n_components = min(X_scaled.shape[0], X_scaled.shape[1])
pca = PCA(n_components=n_components)
pca.fit(X_scaled)
loadings = np.abs(pca.components_.T)
explained_variance_ratio = pca.explained_variance_ratio_
feature_importance = np.dot(loadings, explained_variance_ratio)
print(feature_importance.shape)
feature_ranking_indices = np.argsort(-feature_importance)   # Descending order
selected_features_PCA = X_func.columns[feature_ranking_indices].values
print(selected_features_PCA)

cmap = mcolors.LinearSegmentedColormap.from_list("red_fade", ["red", "white"], N=selected_features_PCA.size)
colors = {feature: cmap(i) for i, feature in enumerate(selected_features_PCA)}
colors_subset = {key:value for index, (key,value) in enumerate(colors.items()) if index>550}

plt.figure(figsize=(20,2))
for count, (feature, color) in enumerate(colors.items()):
    plt.vlines(x=count, ymin=0, ymax=0.5, color=color, linewidth=5)
plt.savefig("feature_selection.png")
plt.show()

In [ ]:
# Lasso Regularization

lasso = Lasso(alpha=0.00001, max_iter=100000)
lasso.fit(X_scaled, Y)
coefficients = lasso.coef_
selected_indices = np.where(coefficients != 0)[0]
selected_features_Lasso = X_func.columns[selected_indices].values
sorted_features_Lasso = np.array([label for label in selected_features_PCA if label in selected_features_Lasso])
plt.figure(figsize=(20,2))
for count,label in enumerate(sorted_features_Lasso):
    plt.vlines(x=count, ymin=0, ymax=0.5, color=colors[label], linewidth=5)
plt.savefig("feature_selection.png")
plt.show()

In [ ]:
# ANOVA

select_k_best = SelectKBest(score_func=f_regression, k=100)
X_selected = select_k_best.fit_transform(X_func, Y)
feature_importance = select_k_best.scores_
selected_indices = select_k_best.get_support(indices=True)
selected_features_ANOVA = X_func.columns[selected_indices].values
feature_importance_dict = {key:value for key,value in zip(selected_features_ANOVA, feature_importance)}
feature_importance_dict = dict(sorted(feature_importance_dict.items(), key=lambda item: -item[1])) # Descending order
print(feature_importance_dict)
plt.figure(figsize=(20,2))
for count,label in enumerate(feature_importance_dict.keys()):
    plt.vlines(x=count, ymin=0, ymax=0.5, color=colors[label], linewidth=5)
plt.savefig("feature_selection.png")
plt.show()

In [ ]:
# MIR

mi = mutual_info_regression(X_func, Y)
mi_df = pd.DataFrame({
    'Feature': X_func.columns,
    'Mutual Information': mi
})
mi_df = mi_df.sort_values(by='Mutual Information', ascending=False)
selected_features_MIR = mi_df['Feature'].values
plt.figure(figsize=(20,2))
for count,label in enumerate(selected_features_MIR):
    plt.vlines(x=count, ymin=0, ymax=0.5, color=colors[label], linewidth=5)
plt.savefig("feature_selection.png")
plt.show()

In [ ]:
# Reduction of Features

selected_features_overall = np.array([feature for feature in selected_features_PCA[0:50] if (feature in selected_features_ANOVA) and (feature in selected_features_MIR[0:50] and (feature in selected_features_Lasso))])
X_reduced = X_func[selected_features_overall[0:20]]
print(X_reduced.columns)

In [ ]:
tpot = TPOTRegressor(generations=70, population_size=70, scoring='r2')
tpot.fit(X_func,Y)


In [30]:
tpot.export('best_ml_algo.py')